In [2]:
#!pip install langchain langchain-community langchain-core transformers==4.52.4
#!pip install fastapi uvicorn pyngrok accelerate -q

### **Importing Libraries**

In [3]:
from fastapi import FastAPI, Request, HTTPException, UploadFile, File
import uvicorn, threading, time, socket, tempfile, re, json
from pyngrok import ngrok, conf
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader

### **Loading Model**

In [4]:
model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

2026-01-11 15:27:48.257712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768145268.305137     307 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768145268.318547     307 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768145268.384767     307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768145268.384838     307 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768145268.384843     307 computation_placer.cc:177] computation placer alr

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### **Helper Functions**

In [5]:
def generate_text(prompt, max_length=2000, num_return_sequences=1):
    """Generate text using the loaded model"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=False,  # Deterministic output
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]


def extract_json_block(text):
    """Extract JSON block from markdown-formatted text"""
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[-1]
    return None


def print_cv_info(cv_data):
    print("\n" + "="*60)
    print("           CV PARSING RESULTS")
    print("="*60 + "\n")
    
    try:
        # Parse JSON if it's a string
        if isinstance(cv_data, str):
            cv_data = json.loads(cv_data)
        
        # Print each field
        print(f"📝 FULL NAME:")
        print(f"   {cv_data.get('full_name', 'Not found')}\n")
        
        print(f"📧 EMAIL:")
        print(f"   {cv_data.get('email', 'Not found')}\n")
        
        print(f"🎓 EDUCATION:")
        education = cv_data.get('education', 'Not found')
        if isinstance(education, list):
            for edu in education:
                print(f"   • {edu}")
        else:
            print(f"   {education}")
        print()
        
        print(f"💼 SKILLS:")
        skills = cv_data.get('skills', 'Not found')
        if isinstance(skills, list):
            for skill in skills:
                print(f"   • {skill}")
        else:
            print(f"   {skills}")
        print()
        
        print(f"🏢 EXPERIENCE:")
        experience = cv_data.get('experience', 'Not found')
        if isinstance(experience, list):
            for exp in experience:
                print(f"   • {exp}")
        else:
            print(f"   {experience}")
        
    except json.JSONDecodeError:
        print("⚠️  Could not parse JSON. Raw output:")
        print(cv_data)
    
    print("\n" + "="*60 + "\n")

### **Defining Schemas**

In [6]:
full_name_schema = ResponseSchema(name="full_name",description="The full name of the person in the CV")
email_schema = ResponseSchema(name="email",description="The email address of the person")
education_schema = ResponseSchema(name="education",description="The education details such as degree, institution, and graduation year")
skills_schema = ResponseSchema(name="skills",description="A list of technical or soft skills mentioned in the CV")
experience_schema = ResponseSchema(name="experience",description="Work experience details including role, company, and years")

response_schemas = [full_name_schema,email_schema,education_schema,skills_schema,experience_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

### **CV Extraction Template**

In [7]:
CV_extraction_template = """
You are an intelligent CV parser. Extract the following information from the provided resume text:

- Full name
- Email
- Education (degree, institution, year)
- Skills
- Experience (role, company, years)

Return the output in JSON format exactly as instructed below.

{format_instructions}

Resume text:
{resume_text}
"""

### **Main Function - Parse CV from File**

In [8]:
def parse_cv_from_file(pdf_path):
 
    print(f"\n🔍 Loading CV from: {pdf_path}")
    
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    resume_text = " ".join([page.page_content for page in pages])
    
    print("📄 PDF loaded successfully")
    print(" Processing with AI model...\n")
    
    # Create prompt
    prompt = PromptTemplate(
        template=CV_extraction_template,
        input_variables=["resume_text", "format_instructions"],
    ).format(resume_text=resume_text, format_instructions=format_instructions)
    
    # Generate output
    raw_output = generate_text(prompt)
    
    # Extract JSON
    json_str = extract_json_block(raw_output)
    
    if json_str:
        try:
            cv_data = json.loads(json_str)
            print_cv_info(cv_data)
            return cv_data
        except json.JSONDecodeError:
            print("⚠️  JSON parsing failed. Showing raw output:")
            print(json_str)
            return {"raw_output": json_str}
    else:
        print("⚠️  No JSON block found in output. Showing raw output:")
        print(raw_output)
        return {"raw_output": raw_output}


### **Test with a Local File**

In [9]:
#  Modify the path to test with your CV
result = parse_cv_from_file("/kaggle/input/datasetparser/Muhammed Waheed .pdf")


🔍 Loading CV from: /kaggle/input/datasetparser/Muhammed Waheed .pdf


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📄 PDF loaded successfully
 Processing with AI model...


           CV PARSING RESULTS

📝 FULL NAME:
   Muhammed Waheed Muhammmed

📧 EMAIL:
   wahedmohamed688@gmail.com

🎓 EDUCATION:
   Bachelor's Degree in Computer Science - Al Major, Cairo University

💼 SKILLS:
   Programming Languages: Python, C++, SQL; Deep Learning & Libraries: PyTorch, TensorFlow, Keras, Scikit-learn, Pandas, NumPy, Seaborn, Matplotlib; Computer Vision: YOLO, Face Recognition, Pose Estimation, Image Inpainting, Variational Autoencoders (VAE); Natural Language Processing (NLP): BERT, RAG, Prompt Engineering, Text Classification, Web Scraping, Text Preprocessing; Generative AI: Large Language Models (LLMs), Retrieval-Augmented Generation (RAG), LangChain; Data Science: Regression, Classification, Time Series Forecasting, Exploratory Data Analysis (EDA), Data Wrangling; Tools & Technologies: Git, Docker, FastAPI, Streamlit, Ngrok; Languages: Arabic (native), English (B1)

🏢 EXPERIENCE:
   Ai Trainee, Tips Hindawi; D

### **API Configuration**

In [10]:
NGROK_TOKEN = "3879vTnZnagyraXGS8aXFFlgbcY_5VBNMHoXbTsZwQPYoz3RF"
API_KEY = "secret123"

### **FastAPI Setup**

In [11]:
app = FastAPI()

@app.post("/parse_cv")
async def parse_cv(file: UploadFile = File(...), req: Request = None):
    """API endpoint to parse CV"""
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    # Save uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(await file.read())
        pdf_path = tmp.name

    # Parse the CV
    try:
        result = parse_cv_from_file(pdf_path)
        return {"success": True, "parsed_cv": result}
    except Exception as e:
        return {"success": False, "error": str(e)}


def free_port():
    """Find a free port"""
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

### **Start API Server**

In [12]:
port = free_port()

conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url

print("\n" + "="*60)
print("           API SERVER STARTED")
print("="*60)
print(f"\n📡 Endpoint: POST {public_url}/parse_cv")
print(f"🔑 API Key: {API_KEY}")
print("\n" + "="*60 + "\n")

def run(): 
    uvicorn.run(app, host="0.0.0.0", port=port)

threading.Thread(target=run, daemon=True).start()
time.sleep(1)


           API SERVER STARTED

📡 Endpoint: POST https://leptophyllous-cachectic-colleen.ngrok-free.dev/parse_cv
🔑 API Key: secret123




INFO:     Started server process [307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:42549 (Press CTRL+C to quit)
